In [ ]:
import gi
gi.require_version('Gst', '1.0')
from gi.repository import Gst
import cv2
import numpy as np
import degirum as dg
import time
import os

In [ ]:
def video_frame_generator(video_source="Traffic.mp4", target_size=(640, 640)):
    """
    Generator that yields frames from a video source using GStreamer.
    Resizes frames to target_size using GStreamer pipeline and yields both resized frame and frame_info.
    
    Args:
        video_source: Video source (path to video file, e.g., "Traffic.mp4")
        target_size: Tuple of (width, height) for resizing
    
    Yields:
        tuple: (resized_frame, frame_info)
            - resized_frame: Frame resized to target_size via GStreamer
            - frame_info: Dict with 'original_frame', 'original_width', 'original_height'
    """
    # Initialize GStreamer
    Gst.init(None)
    
    # First, get original dimensions using OpenCV (one-time check)
    cap = cv2.VideoCapture(video_source)
    if not cap.isOpened():
       raise ValueError(f"Could not open video source: {video_source}")
    
    ret, sample_frame = cap.read()
    if not ret:
       cap.release()
       raise ValueError("Could not read sample frame to get dimensions")
    
    original_height, original_width = sample_frame.shape[:2]
    cap.release()

    print(f"Original video dimensions: {original_width}x{original_height}")
    print(f"Target size: {target_size}")
    
    # Create GStreamer pipeline for video file with tee to split into original and resized streams
    target_width, target_height = target_size
    pipeline_str = (
        f"filesrc location={video_source} ! "
        f"decodebin ! videoconvert ! video/x-raw, format=BGR ! tee name=t "
        f"t. ! queue ! appsink name=original_sink emit-signals=true max-buffers=1 drop=true "
        f"t. ! queue ! videoscale ! video/x-raw, format=BGR, width={target_width}, height={target_height} ! "
        f"appsink name=resized_sink emit-signals=true max-buffers=1 drop=true"
    )
    
    pipeline = Gst.parse_launch(pipeline_str)
    original_sink = pipeline.get_by_name("original_sink")
    resized_sink = pipeline.get_by_name("resized_sink")
    
    pipeline.set_state(Gst.State.PLAYING)
    
    try:
        while True:
            # Get resized frame
            resized_sample = resized_sink.emit("pull-sample")
            if not resized_sample:
                break
                
            # Get original frame
            original_sample = original_sink.emit("pull-sample")
            if not original_sample:
                break
            
            # Process resized frame
            resized_buf = resized_sample.get_buffer()
            resized_caps = resized_sample.get_caps()
            resized_width = resized_caps.get_structure(0).get_value("width")
            resized_height = resized_caps.get_structure(0).get_value("height")
            
            success, resized_map_info = resized_buf.map(Gst.MapFlags.READ)
            if not success:
                continue
                
            resized_frame = np.frombuffer(resized_map_info.data, np.uint8).reshape((resized_height, resized_width, 3))
            resized_buf.unmap(resized_map_info)
            
            # Process original frame
            original_buf = original_sample.get_buffer()
            original_caps = original_sample.get_caps()
            orig_width = original_caps.get_structure(0).get_value("width")
            orig_height = original_caps.get_structure(0).get_value("height")
            
            success, original_map_info = original_buf.map(Gst.MapFlags.READ)
            if not success:
                continue
                
            original_frame = np.frombuffer(original_map_info.data, np.uint8).reshape((orig_height, orig_width, 3))
            original_buf.unmap(original_map_info)
            
            # Create frame_info dictionary
            frame_info = {
                'original_frame': original_frame.copy(),
                'original_width': orig_width,
                'original_height': orig_height
            }
            
            yield resized_frame, frame_info
                
    finally:
        pipeline.set_state(Gst.State.NULL)


In [ ]:
class CustomPostprocessor(dg.postprocessor.DetectionResults):
    """
    Custom postprocessor that plots results on the original frame using frame_info.
    Frame info can be accessed via results.info.
    """
    
    def __init__(self, input_image, model_image, inference_results, **kwargs):
        # Initialize parent class
        super().__init__(
            input_image=input_image,
            model_image=model_image, 
            inference_results=inference_results,
            **kwargs
        )
    
    def plot_results_on_original(self):
        """
        Plot detection results on the original frame (not resized frame).
        Uses frame_info from results.info to get original frame and dimensions.
        """
        # Access frame_info from results.info
        if not hasattr(self, 'info') or self.info is None:
            print("Warning: frame_info not available in results.info")
            return self.input_image
        
        frame_info = self.info
        original_frame = frame_info['original_frame'].copy()
        original_width = frame_info['original_width']
        original_height = frame_info['original_height']
        
        # Get detection results
        detections = self.results
        
        # Calculate scaling factors from resized (640x640) to original dimensions
        scale_x = original_width / 640
        scale_y = original_height / 640
        
        # Plot each detection on the original frame
        for detection in detections:
            if 'bbox' in detection:
                # Scale bounding box coordinates back to original frame size
                x1, y1, x2, y2 = detection['bbox']
                x1 = int(x1 * scale_x)
                y1 = int(y1 * scale_y) 
                x2 = int(x2 * scale_x)
                y2 = int(y2 * scale_y)
                
                # Draw bounding box
                cv2.rectangle(original_frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                
                # Draw label if available
                if 'label' in detection:
                    label = detection['label']
                    score = detection.get('score', 0)
                    text = f"{label}: {score:.2f}"
                    
                    # Calculate text size and position
                    (text_width, text_height), baseline = cv2.getTextSize(
                        text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
                    )
                    
                    # Draw text background
                    cv2.rectangle(
                        original_frame,
                        (x1, y1 - text_height - baseline - 5),
                        (x1 + text_width, y1),
                        (0, 255, 0), -1
                    )
                    
                    # Draw text
                    cv2.putText(
                        original_frame,
                        text,
                        (x1, y1 - baseline - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.6,
                        (0, 0, 0),
                        2
                    )
        
        return original_frame
    
    @property
    def image_overlay(self):
        """Override image_overlay to return the original frame with detections plotted."""
        return self.plot_results_on_original()



In [ ]:
def main(output_dir="result_frames", max_frames=100):
    """
    Main function demonstrating the video generator and custom postprocessor.
    Saves result frames to a directory instead of displaying them.
    
    Args:
        output_dir: Directory to save result frames
        max_frames: Maximum number of frames to process (0 for unlimited)
    """
    # Create output directory if it doesn't exist
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Created output directory: {output_dir}")
    else:
        print(f"Saving frames to existing directory: {output_dir}")
    
    # Load your model (adjust model_name and other parameters as needed)
    try:
        model = dg.load_model(
            model_name="yolov8n_coco--640x640_quant_hailort_multidevice_1",
            inference_host_address="@local",
            zoo_url="degirum/hailo"
        )
    except Exception as e:
        print(f"Error loading model: {e}")
        print("Please adjust the model parameters for your setup")
        return
    
    # Attach the custom postprocessor
    model.custom_postprocessor = CustomPostprocessor
    
    # Warmup inference
    print("Performing warmup inference...")
    dummy_frame = np.zeros((640, 640, 3), dtype=np.uint8)
    _ = model.predict(dummy_frame)
    print("Warmup complete")
    
    # Create a wrapper generator that provides frame_info in the correct format
    def model_input_generator():
        for resized_frame, frame_info in video_frame_generator(video_source="Traffic.mp4"):
            # Yield the resized frame for model inference
            # The frame_info will be attached to results.info
            yield resized_frame, frame_info
    
    # FPS tracking
    frame_count = 0
    saved_frames = 0
    start_time = time.time()
    
    print(f"Starting video inference... Will save up to {max_frames if max_frames > 0 else 'unlimited'} frames")
    
    try:
        # Run inference on video frames
        for result in model.predict_batch(model_input_generator()):
            # The frame_info is now available in result.info
            # The custom postprocessor will use it to plot on original frame
            
            frame_count += 1
            
            # Save the result frame (original frame with detections)
            frame_filename = os.path.join(output_dir, f"frame_{saved_frames:06d}.jpg")
            success = cv2.imwrite(frame_filename, result.image_overlay)
            
            if success:
                saved_frames += 1
                if saved_frames % 10 == 0:  # Print progress every 10 saved frames
                    print(f"Saved {saved_frames} frames")
            else:
                print(f"Failed to save frame {saved_frames}")
            
            elapsed_time = time.time() - start_time
            
            # Print FPS every second
            if elapsed_time >= 1.0:
                fps = frame_count / elapsed_time
                print(f"Processing FPS: {fps:.2f} | Saved frames: {saved_frames}")
                frame_count = 0
                start_time = time.time()
            
            # Stop if max_frames reached
            if max_frames > 0 and saved_frames >= max_frames:
                print(f"Reached maximum frames limit: {max_frames}")
                break
                
    except KeyboardInterrupt:
        print("Interrupted by user")
    finally:
        print(f"Total frames saved: {saved_frames}")
        print(f"Frames saved in directory: {output_dir}")


if __name__ == "__main__":
    main()
